In [2]:
import rasterio as rio
import numpy as np 
import pathlib
import PIL
from PIL import Image
import sys
import xml.etree.cElementTree as ET
from itertools import product

In [31]:
path = pathlib.Path("./smallareatiled/classified20")

imgs = [ PIL.Image.open(p.absolute()) for p in path.rglob("*") if p.name.endswith(".png") ]

v_stacks = []
v_stacks_min_max = []

# By default gdal2tiles stores the tiles in the following manner:
# tiles
# ├ <zoom-level-1>
# |      ├ <column-level-1>
# |      |      ├ <row-level-1>.png
# |      |      ├ <row-level-...>.png
# |      |      └ <row-level-n>.png
# |      ├ <column-level-...>
# |      |      ├ <row-level-1>.png
# |      |      ├ <row-level-...>.png
# |      |      └ <row-level-n>.png
# |      ├ <column-level-m>
# |      |      ├ <row-level-1>.png
# |      |      ├ <row-level-...>.png
# |      |      └ <row-level-n>.png
# ...
# 
# Exemplary tile-format
# col-<...>:
#________1__2__3__4____________________
# row-4 |▐█|██|██|
# row-3 | █|██|██|▌
# row-2 | ▐|██|██|█
# row-1 |  |██|██|█▌
#
# Note that <col-1, row-1> and <col-4, row-4> will not have an image stored. 
# Hence we need to fill up those empty spaces.
# 
# For the machine learing process those tile have been merged to be in only one directory:
# i: 1 -> n, j: 1 -> m
# tiles_merged
# ├ <column-level-1>-<row-level-1>.png
# ├ <column-level-i>-<row-level-j>.png
# └ <column-level-m>-<row-level-n>.png

prefix_before = ""
for img in imgs:
    name = pathlib.Path(img.filename).name
    prefix = name[:name.index("-")]
    suffix = name[name.index("-")+1:name.index(".png")]
    
    new_column = not prefix_before == prefix

    if new_column:
        v_stacks.append([])
        v_stacks_min_max.append([ sys.maxsize, -sys.maxsize ])

    v_stacks[len(v_stacks)-1].append(np.asarray(img))

    current_min = v_stacks_min_max[len(v_stacks)-1][0]
    current_max = v_stacks_min_max[len(v_stacks)-1][1]

    v_stacks_min_max[len(v_stacks)-1] = [min(current_min, int(suffix)), max(current_max, int(suffix))]

    prefix_before = prefix

start = min([ min_max[0] for min_max in v_stacks_min_max ])
end = max([ min_max[1] for min_max in v_stacks_min_max ])

for i, stack in enumerate(v_stacks):
    column_start = v_stacks_min_max[i][0]
    column_end = v_stacks_min_max[i][1]

    missing_lower = column_start - start
    missing_upper = end - column_end

    shape = stack[0].shape

    lower_stack = np.zeros((missing_lower, shape[0], shape[1], shape[2]), dtype="uint8")
    upper_stack = np.zeros((missing_upper, shape[0], shape[1], shape[2]), dtype="uint8")

    filled_stack = np.vstack([lower_stack, stack, upper_stack])
    filled_stack = np.vstack(np.flipud(filled_stack))

    v_stacks[i] = filled_stack

v_stacks = np.asarray(v_stacks)
merged = np.hstack(v_stacks)
PIL.Image.fromarray(merged).save(path.name + ".png")

In [22]:
# A rather tricky part about this whole process is that gdal2tiles creates tiles of a fixed size.
# In 99.99% of all cases this will lead to the case that parts of the outermost tiles in any direction will be filled with transparent pixels,
# since the original tif will have some form of rotation/tilt.
# When we want to inversively georeference the resulted tiles, we have to get rid of those empty spaces. So we will have to find what I called "crop-bounds"

# Exemplary tile-format
# col-<...>:
#________1__2__3__4____________________
#       |░░░░░░░░░░░░░
# row-4 |░▐█|██|██|░░░
# row-3 |░░█|██|██|▌░░
# row-2 |░░▐|██|██|█░░
# row-1 |░░░|██|██|█▌░
#       |░░░░░░░░░░░░░
#
# ░ -> empty spaces

In [26]:
crop_bounds = {}

for y, x in product(range(merged.shape[0]), range(merged.shape[1])):
    if not np.array_equal(merged[y][x], [0, 0, 0, 0]):
        print("First pixel non transparent pixel from top at pos", x, y)
        crop_bounds["top"] = y
        break

for y, x in product(reversed(range(merged.shape[0])), range(merged.shape[1])):
    if not np.array_equal(merged[y][x], [0, 0, 0, 0]):
        print("First pixel non transparent pixel from bot at pos", x, y)
        crop_bounds["bot"] = y
        break

for x, y in product(range(merged.shape[1]), range(merged.shape[0])):
    if not np.array_equal(merged[y][x], [0, 0, 0, 0]):
        print("First pixel non transparent pixel from left at pos", x, y)
        crop_bounds["left"] = x
        break

for x, y in product(reversed(range(merged.shape[1])), range(merged.shape[0])):
    if not np.array_equal(merged[y][x], [0, 0, 0, 0]):
        print("First pixel non transparent pixel from right at pos", x, y)
        crop_bounds["right"] = x
        break

cropped = merged[crop_bounds["top"]:crop_bounds["bot"], crop_bounds["left"]:crop_bounds["right"]]
PIL.Image.fromarray(cropped).save(path.name + "_classified_cropped.png")

First pixel non transparent pixel from top at pos 185 117


In [20]:
# Reproject the stitched tiles

from pyproj import Proj, transform

P3857 = Proj(init='epsg:3857')
P4326 = Proj(init='epsg:4326')

tree = ET.ElementTree(file="C:/Users/mathias/Documents/Sync/Master/sem3/P2/smallareatiled" + "/tilemapresource.xml")
output_file_path = "smallareatiled_classified_classified_cropped.tif"

bb = tree.getroot().find("BoundingBox").attrib
srs = tree.getroot().find("SRS").text

# Convert string values to numerics
bb = {k: float(v) for k, v in bb.items()}
minx,miny = transform(P4326, P3857, bb["minx"], bb["miny"])
maxx,maxy = transform(P4326, P3857, bb["maxx"], bb["maxy"])

unreferenced = rio.open("smallareatiled_classified_classified_cropped.png")
bands = [1, 2, 3]#, 4]
data = unreferenced.read(bands)

t = rio.transform.from_bounds(
    minx, miny, maxx, maxy, data.shape[1], data.shape[2]
)

with rio.open(
    output_file_path,
    'w',
    driver='GTiff',
    width=data.shape[1],
    height=data.shape[2],
    count=3, 
    nodata=0,
    dtype=data.dtype, 
    transform=t,
    crs=srs) as dst:
        dst.write(data, indexes=bands)


In [ ]:
# Directly from a gdal2tiles directory (where paths have not been altered previously)
path = pathlib.Path("D:/boku/geodata/Nagycenk/tiled/22")

v_stacks = []
v_stacks_min_max = []

# By default gdal2tiles stores the tiles in the following manner:
# tiles
# ├ <zoom-level-1>
# |      ├ <column-level-1>
# |      |      ├ <row-level-1>.png
# |      |      ├ <row-level-...>.png
# |      |      └ <row-level-n>.png
# |      ├ <column-level-...>
# |      |      ├ <row-level-1>.png
# |      |      ├ <row-level-...>.png
# |      |      └ <row-level-n>.png
# |      ├ <column-level-m>
# |      |      ├ <row-level-1>.png
# |      |      ├ <row-level-...>.png
# |      |      └ <row-level-n>.png
# ...
# 
# Exemplary tile-format
# col-<...>:
#________1__2__3__4____________________
# row-4 |▐█|██|██|
# row-3 | █|██|██|▌
# row-2 | ▐|██|██|█
# row-1 |  |██|██|█▌
#
# Note that <col-1, row-1> and <col-4, row-4> will not have an image stored. 
# Hence we need to fill up those empty spaces.

for p in path.rglob("*"):
    if p.name.endswith(".png"):
        with PIL.Image.open(p.absolute()) as img:
            name = p.as_posix()
            prefix = name[:name.rfind("/")]
            prefix = prefix[prefix.rfind("/")+1:]
            suffix = name[name.rfind("/")+1:name.index(".png")]
            
            new_column = not prefix_before == prefix

            if new_column:
                v_stacks.append([])
                v_stacks_min_max.append([ sys.maxsize, -sys.maxsize ])

            v_stacks[len(v_stacks)-1].append(np.asarray(img))

            current_min = v_stacks_min_max[len(v_stacks)-1][0]
            current_max = v_stacks_min_max[len(v_stacks)-1][1]

            v_stacks_min_max[len(v_stacks)-1] = [min(current_min, int(suffix)), max(current_max, int(suffix))]

            prefix_before = prefix

start = min([ min_max[0] for min_max in v_stacks_min_max ])
end = max([ min_max[1] for min_max in v_stacks_min_max ])

for i, stack in enumerate(v_stacks):
    column_start = v_stacks_min_max[i][0]
    column_end = v_stacks_min_max[i][1]

    missing_lower = column_start - start
    missing_upper = end - column_end

    shape = stack[0].shape

    lower_stack = np.zeros((missing_lower, shape[0], shape[1], shape[2]), dtype="uint8")
    upper_stack = np.zeros((missing_upper, shape[0], shape[1], shape[2]), dtype="uint8")

    filled_stack = np.vstack([lower_stack, stack, upper_stack])
    filled_stack = np.vstack(np.flipud(filled_stack))

    v_stacks[i] = filled_stack

v_stacks = np.asarray(v_stacks)
merged = np.hstack(v_stacks)
PIL.Image.fromarray(merged).save(path.name + "other.png")